In [1]:
!pip install indic-nlp-library
!pip install torch
!pip install torchtext
!pip install spacy
!pip install torchdata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 40 kB 1.6 MB/s 
     |████████████████████████████████| 2.8 MB 8.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 47 kB 3.3 MB/s 
     |████████████████████████████████| 138 kB 15.3 MB/s 
     |████████████████████████████████| 127 kB 61.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchtext
from torchtext import data
from torchtext import datasets

from collections import Counter
from torchtext.vocab import vocab

import numpy as np
import random

In [3]:
import sys
import os
from google.colab import drive 
drive.mount('/content/gdrive', force_remount=True)
%cd /content/gdrive/MyDrive/AML_Project/
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/AML_Project"

Mounted at /content/gdrive
/content/gdrive/MyDrive/AML_Project


In [4]:
import pandas as pd
import re

df = pd.DataFrame(columns=['sentence'])
with open('/content/gdrive/MyDrive/AML_Project/tagger_data/newTrainH1.txt',encoding="utf8") as fp:
   line = fp.readline()
   cnt = 0
   while line:
       #print("Line {}: {}".format(cnt, line.strip()))
       line = fp.readline()
       line = line.replace('_', '')
       df.loc[cnt, ['sentence']] = line
       cnt += 1 
       

def split_for_tagger(sentence):
    res = re.findall(r'\[.*?\]', sentence)
    RES= []
    for elem in res:
        #print(elem)
        a = elem.replace('[','').replace(']','')
        b = a.replace(" ","")
        RES.append(b)
    return RES

def split_for_word(sentence):
    string = re.sub("\[.*?\]","",sentence)
    c= string.split()
    return c
    
df['pos'] = df['sentence'].apply(split_for_tagger)
df['clean_sentence'] = df['sentence'].apply(split_for_word)
    
df.drop(['sentence'], axis=1, inplace= True)


In [5]:
df

,pos,clean_sentence
0,"[JJ, NN, PSP, NN, SYM, PRP, NN, SYM, NN, PSP, ...","[पुरानी, पीढ़ी, के, भारतीय, --, हमारे, पिता-पित..."
1,"[JJ, SYM, JJ, SYM, NST, VM, PSP, JJ, NN, PSP, ...","[बदमिज़ाज, ,, चिड़चिड़े, ,, दूर-दूर, रहने, वाले, ..."
2,"[PRP, NN, PSP, INTF, JJ, NN, PRP, NN, VM, VAUX...","[अपनी, ऐंठ, में, थोड़ा-बहुत, भद्र, पीढ़ी-दर-पीढ़ी..."
3,"[PRP, NN, VM, CC, CC, PRP, SYM, NN, SYM, CC, S...","[मुझे, विश्वास, है, कि, अगर, उनकी, `, खुशमिज़ाज..."
4,"[CC, PRP, PRP, PRP, NN, PSP, NN, VM, VAUX, VAU...","[और, जब, मैं, अपने, पिता, की, बात, कर, रहा, हू..."
...,...,...
2595,"[NNP, PSP, VM, SYM, SYM, NNP, CC, NNP, JJ, JJ,...","[शिराक़, ने, कहा, ,, "", यूरोप, और, अमरीका, सच्..."
2596,"[PRP, JJ, NN, PSP, QO, NN, NN, NNP, PSP, NNP, ...","[अपने, यूरोपीय, दौरे, के, दूसरे, दिन, राष्ट्रप..."
2597,"[NNP, PSP, DEM, NNP, VM, PRP, DEM, NN, NNP, NN...","[बुधवार, को, वे, जर्मनी, जाएँगे, जहाँ, वे, चां..."
2598,"[NN, PSP, JJ, NN, NNP, PSP, NNP, NNP, NNP, VM,...","[दौरे, के, अंतिम, दिन, गुरूवार, को, जॉर्ज, बुश..."


### Have to convert it into iterator; Not very sure how to go about it!


In [ ]:
# It is important to recreate the training set afresh 
# each time they are used, since they are Python iterators, and
# once use, cannot be reused.  One could, however, cast them into
# a list, which would store them permanently.

train_iter_0 = datasets.UDPOS(split = 'train')    
train_iter_vocab = preprocess_data_seq(train_iter_0, 1)

counter_words = Counter()
counter_ud = Counter()
counter_ptb = Counter()
for (text, pos_ud, pos_ptb) in train_iter_vocab:
    counter_words.update(text)
    counter_ud.update(pos_ud)
    counter_ptb.update(pos_ptb)
vocab_words = vocab(counter_words,  specials = ['<unk>'], 
                    special_first = True)    
vocab_words.set_default_index(0)
vocab_ud = vocab(counter_ud)
vocab_ptb = vocab(counter_ptb)